In [4]:
###########################################################################
#        Máster en Ingeniería Informática - Universidad de Valladolid     #
#                                                                         #
#                           Trabajo Fin de Máster                         #
#                                                                         #
#   Implementación de técnicas de RAG (Retrieval Augmented Generation)    #
#   sobre LLM (Large Language Models) para la extracción y generación     #
#                de documentos en las Entidades Públicas                  #
#                                                                         #
#            Cuaderno Jupyter auxiliar para el cálculo de métricas        #
#                                                                         #
#                 Realizado por Miguel Ángel Collado Alonso               #
#                                                                         #
###########################################################################



!pip install rouge_score


# Importamos las librerías necesarias
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import re
import unicodedata
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import pandas as pd



# Función para preprocesar el texto
def preprocesar_texto(texto):

    # Eliminar saltos de línea y caracteres especiales
    texto_limpio = texto.replace("\n", " ").strip()  # Elimina saltos de línea y espacios adicionales
    # Normaliza el texto para separar los caracteres base de los acentos
    texto_limpio = unicodedata.normalize('NFKD', texto_limpio)
    # Filtra solo los caracteres que no son marcas de acento
    texto_limpio = ''.join(c for c in texto_limpio if not unicodedata.combining(c))
    texto_limpio = re.sub(r'\s+', ' ', texto_limpio)  # Reemplaza múltiples espacios por uno solo
    return texto_limpio

# Función para obtener las respuestas a través de un área de texto
def obtener_respuestas():

    # Crea dos áreas de texto grandes para que el usuario introduzca el texto del chatbot y la respuesta correcta
    chatbot_area = widgets.Textarea(
        value='',
        placeholder='Introduce la respuesta generada por el chatbot aquí...',
        description='Chatbot:',
        layout=widgets.Layout(width='100%', height='200px')
    )

    referencia_area = widgets.Textarea(
        value='',
        placeholder='Introduce la respuesta de referencia aquí...',
        description='Referencia:',
        layout=widgets.Layout(width='100%', height='200px')
    )

    # Botón para confirmar la entrada
    button = widgets.Button(description="Calcular Métricas")

    # Muestra las áreas de texto
    display(chatbot_area, referencia_area, button)

    # Función para manejar el evento del botón
    def on_button_clicked(b):
        chatbot_respuesta = preprocesar_texto(chatbot_area.value)
        respuesta_correcta = preprocesar_texto(referencia_area.value)
        calcular_metricas(chatbot_respuesta, respuesta_correcta)

    # Asocia el clic del botón con la función
    button.on_click(on_button_clicked)

# Función para calcular ROUGE
def calcular_rouge(chatbot_respuesta, respuesta_correcta):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(respuesta_correcta, chatbot_respuesta)
    return scores


# Función para calcular precisión, recall y F1 considerando diferencias de longitud
def calcular_exactitud_f1(chatbot_respuesta, respuesta_correcta):
    palabras_chatbot = set(chatbot_respuesta.split())
    palabras_referencia = set(respuesta_correcta.split())

    # Intersección entre las palabras de ambas respuestas (palabras comunes)
    palabras_comunes = palabras_chatbot.intersection(palabras_referencia)

    # Cálculo de precisión, recall y F1
    if len(palabras_chatbot) > 0 and len(palabras_referencia) > 0:
        precision = len(palabras_comunes) / len(palabras_chatbot)
        recall = len(palabras_comunes) / len(palabras_referencia)
        if precision + recall > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
        else:
            f1 = 0.0
    else:
        precision, recall, f1 = 0.0, 0.0, 0.0

    return precision, recall, f1

# Función para calcular similitud coseno
def calcular_similitud_coseno(chatbot_respuesta, respuesta_correcta):

    # Usa CountVectorizer para convertir el texto en vectores
    vectorizador = CountVectorizer().fit_transform([chatbot_respuesta, respuesta_correcta])
    vectores = vectorizador.toarray()

    # Calcula la similitud coseno entre los dos vectores
    similitud = cosine_similarity([vectores[0]], [vectores[1]])

    return similitud[0][0]

# Función principal que calcula las métricas y las muestra en formato de tabla
def calcular_metricas(chatbot_respuesta, respuesta_correcta):

    # Calcula ROUGE
    rouge_scores = calcular_rouge(chatbot_respuesta, respuesta_correcta)

    # Calcula precisión y F1 para respuestas exactas
    precision, recall, f1 = calcular_exactitud_f1(chatbot_respuesta, respuesta_correcta)

    # Calcula similitud coseno
    similitud_coseno = calcular_similitud_coseno(chatbot_respuesta, respuesta_correcta)

    # Muestra las métricas en formato de tabla con pandas
    metricas = {
        "ROUGE-1": [round(rouge_scores["rouge1"].fmeasure, 2)],
        "ROUGE-2": [round(rouge_scores["rouge2"].fmeasure, 2)],
        "ROUGE-L": [round(rouge_scores["rougeL"].fmeasure, 2)],
        "Precisión": [round(precision, 2)],
        "Recall": [round(recall, 2)],
        "F1-Score": [round(f1, 2)],
        "Similitud Coseno": [round(similitud_coseno, 2)]
     }


    df_metricas = pd.DataFrame(metricas)
    display(df_metricas)

# Llama a la función para obtener respuestas
obtener_respuestas()


Textarea(value='', description='Chatbot:', layout=Layout(height='200px', width='100%'), placeholder='Introduce…

Textarea(value='', description='Referencia:', layout=Layout(height='200px', width='100%'), placeholder='Introd…

Button(description='Calcular Métricas', style=ButtonStyle())

,ROUGE-1,ROUGE-2,ROUGE-L,Precisión,Recall,F1-Score,Similitud Coseno
0,0.57,0.4,0.5,0.45,0.84,0.59,0.61


,ROUGE-1,ROUGE-2,ROUGE-L,Precisión,Recall,F1-Score,Similitud Coseno
0,0.56,0.38,0.5,0.41,0.69,0.51,0.67


,ROUGE-1,ROUGE-2,ROUGE-L,Precisión,Recall,F1-Score,Similitud Coseno
0,0.45,0.25,0.41,0.5,0.5,0.5,0.46


,ROUGE-1,ROUGE-2,ROUGE-L,Precisión,Recall,F1-Score,Similitud Coseno
0,0.5,0.23,0.41,0.46,0.53,0.49,0.52
